CASOS CONFIRMADOS

In [1]:
import math # para calcular dados trigonométricos
from typing import Iterator
from datetime import datetime, timedelta # para trabalhar com datas e horas

import numpy as np
import pandas as pd


Processando os dados de casos da universidade John Hopkins.

In [2]:
cases= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')
cases.to_csv('john_hopkins', index=False)

In [3]:
display(cases)


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4007,NaN,NaN,Unknown,Ukraine,2021-01-13 05:22:15,NaN,NaN,0,0,0,0,"Unknown, Ukraine",0.000000,0.000000
4008,NaN,NaN,NaN,Nauru,2021-01-13 05:22:15,-0.52280,166.931500,0,0,0,0,Nauru,0.000000,0.000000
4009,NaN,NaN,Niue,New Zealand,2021-01-13 05:22:15,-19.05440,-169.867200,0,0,0,0,"Niue, New Zealand",0.000000,0.000000
4010,NaN,NaN,NaN,Tuvalu,2021-01-13 05:22:15,-7.10950,177.649300,0,0,0,0,Tuvalu,0.000000,0.000000


O arquivo está copilado em um arquivo por dia.


Portanto, precisaremos iterar dentro de um intervalo de tempo definido para extraí-lo.

In [4]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
    date_range_days: int = (end_date - start_date).days
    for lag in range(date_range_days):
        yield start_date + timedelta(lag)
        

In [5]:
start_date = datetime(2021, 1, 1)
end_date = datetime(2021, 12, 31) 

De maneira iterativa, vamos selecionar as colunas de interesse e as linhas referentes ao Brasil.

In [7]:
cases = []

for date in date_range(start_date=start_date, end_date=end_date):
    date_str = date.strftime('%m-%d-%Y')
    data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'
    
    case = pd.read_csv(data_source_url, sep=',')
    
    case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
    case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
    case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))
    
    cases.append(case)

# Concatenar todos os DataFrames juntos
result = pd.concat(cases, ignore_index=True)

# Agora 'result' contém todos os dados concatenados

In [8]:
result.query('Province_State == "Sao Paulo"').head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
24,Sao Paulo,Brazil,1466191,46775,3192.990778,2021-01-01
51,Sao Paulo,Brazil,1467953,46808,3196.827966,2021-01-02
78,Sao Paulo,Brazil,1471422,46845,3204.382565,2021-01-03
105,Sao Paulo,Brazil,1473670,46888,3209.278136,2021-01-04
132,Sao Paulo,Brazil,1486551,47222,3237.329676,2021-01-05


Manipulação dos dados para o dashboard

In [9]:
cases=result

In [10]:
display(cases)

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01
...,...,...,...,...,...,...
9823,Roraima,Brazil,128793,2078,21261.355551,2021-12-30
9824,Santa Catarina,Brazil,1242654,20183,17343.904663,2021-12-30
9825,Sao Paulo,Brazil,4455011,155186,9701.879932,2021-12-30
9826,Sergipe,Brazil,278507,6057,12115.869171,2021-12-30


In [11]:
cases.shape

(9828, 6)

In [12]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9828 non-null   object        
 1   Country_Region  9828 non-null   object        
 2   Confirmed       9828 non-null   int64         
 3   Deaths          9828 non-null   int64         
 4   Incident_Rate   9828 non-null   float64       
 5   Date            9828 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 460.8+ KB


Começando com o nome das colunas

In [13]:
cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

Ajustando o nome dos estados

In [14]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

Criando Chaves temporais para enriquecer a base de dados

In [15]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year'] = cases['date'].apply(lambda date: date.strftime('%Y'))

População estimada do estado

In [17]:
cases['population'] = round(100000* (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

In [18]:
# Definição da função get_trend
def get_trend(rate: float) -> str:
    if np.isnan(rate):
        return np.NaN
    
    if rate < 0.75:
        status = 'downward'  # queda
    elif rate > 1.15:
        status = 'upward'     
    else:
        status = 'stable'
    return status

In [19]:
cases_ = []  # lista para armazenar os DataFrames
cases_is_empty = True  
for state in cases['state'].drop_duplicates():
    cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
    cases_per_state = cases_per_state.sort_values(by=['date'])

    cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
    cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
    cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d'] / cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
    cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

    cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
    cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
    cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d'] / cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
    cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

    if cases_is_empty:  
        cases_ = cases_per_state.copy()  # Usei copy para criar um novo DataFrame
        cases_is_empty = False
    else:
        cases_ = pd.concat([cases_, cases_per_state], ignore_index=True)  # Concatene DataFrames

In [26]:
display(cases_)

,state,country,confirmed,deaths,date,month,year,population,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend
0,Acre,Brazil,41689,796,2021-01-01,2021-01,2021,881935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acre,Brazil,41941,798,2021-01-02,2021-01,2021,881935.0,252.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
2,Acre,Brazil,42046,802,2021-01-03,2021-01,2021,881935.0,105.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
3,Acre,Brazil,42117,806,2021-01-04,2021-01,2021,881935.0,71.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
4,Acre,Brazil,42170,808,2021-01-05,2021-01,2021,881935.0,53.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9823,Tocantins,Brazil,234113,3927,2021-12-26,2021-12,2021,1572866.0,0.0,0.0,0.000000,downward,0.0,0.0,0.0,downward
9824,Tocantins,Brazil,234113,3927,2021-12-27,2021-12,2021,1572866.0,0.0,0.0,0.000000,downward,0.0,0.0,0.0,downward
9825,Tocantins,Brazil,234964,3933,2021-12-28,2021-12,2021,1572866.0,851.0,122.0,2.837209,upward,6.0,1.0,1.0,stable
9826,Tocantins,Brazil,235340,3936,2021-12-29,2021-12,2021,1572866.0,376.0,176.0,inf,upward,3.0,2.0,inf,upward


Garantir o tipo do dado é fundamental para consistência da base de dados. Vamos fazer o *type casting* das colunas.

In [27]:
cases = cases_
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

Por fim, vamos reorganizar as colunas e conferir o resultado final.

In [28]:
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

In [29]:
cases.head(n=25)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,2021-01,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,<NA>,NaN,NaN,798,2,<NA>,NaN,NaN,2021-01,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,<NA>,NaN,NaN,802,4,<NA>,NaN,NaN,2021-01,2021
3,2021-01-04,Brazil,Acre,881935,42117,71,<NA>,NaN,NaN,806,4,<NA>,NaN,NaN,2021-01,2021
4,2021-01-05,Brazil,Acre,881935,42170,53,<NA>,NaN,NaN,808,2,<NA>,NaN,NaN,2021-01,2021
5,2021-01-06,Brazil,Acre,881935,42378,208,<NA>,NaN,NaN,814,6,<NA>,NaN,NaN,2021-01,2021
6,2021-01-07,Brazil,Acre,881935,42478,100,<NA>,NaN,NaN,821,7,<NA>,NaN,NaN,2021-01,2021
7,2021-01-08,Brazil,Acre,881935,42814,336,161,NaN,NaN,823,2,4,NaN,NaN,2021-01,2021
8,2021-01-09,Brazil,Acre,881935,42908,94,139,NaN,NaN,823,0,4,NaN,NaN,2021-01,2021
9,2021-01-10,Brazil,Acre,881935,43127,219,155,NaN,NaN,825,2,4,NaN,NaN,2021-01,2021


Salvando o arquivo

In [30]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)